### Project: 'MODELO PREDICTIVO DE CLASIFICACIÓN DE CHURN BASADO EN PATRONES DE CONTACTO PARA UNA STARTUP BOLIVIANA'  
#### Author: Flavia Davila Perez
#### Arquitecture: Medallion
##### Stage: Silver 
##### Sub-Stage: Enriched
Description: Data transformation: pivoting, merging, calculation of new columns


### Libraries

In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta

In [2]:
import sys

sys.path.append('../scripts')
from silver_functions import *

### Import Filtered Data

In [3]:
# Import Filtered Data from CSV

# Activities
df_activities_com = pd.read_csv(r'..\data\output_silver\02_silver_filtered\activities_com.csv')
df_activities_exp = pd.read_csv(r'..\data\output_silver\02_silver_filtered\activities_exp.csv')

# Deals
df_deals_com = pd.read_csv(r'..\data\output_silver\02_silver_filtered\deals_com.csv')
df_deals_exp = pd.read_csv(r'..\data\output_silver\02_silver_filtered\deals_exp.csv')

#### Drop Duplicates

In [ ]:
df_deals_com = df_deals_com[df_deals_com.duplicated(subset='org_id.value', keep=False) == False]
df_deals_exp = df_deals_exp[df_deals_exp.duplicated(subset='org_id.value', keep=False) == False]

### Columns Combination

In [5]:
# new column "Fecha y Hora de vencimiento" - COM
df_activities_com["Fecha y Hora de vencimiento"] = (
    df_activities_com["Fecha de vencimiento"].astype(str) + " " +
    df_activities_com["Hora de vencimiento"].astype(str)
)

# drop old columns
df_activities_com = df_activities_com.drop(
    ["Fecha de vencimiento", "Hora de vencimiento"], axis=1
)

In [6]:
# new column "Fecha y Hora de vencimiento" - EXP
df_activities_exp["Fecha y Hora de vencimiento"] = (
    df_activities_exp["Fecha de vencimiento"].astype(str) + " " +
    df_activities_exp["Hora de vencimiento"].astype(str)
)

# drop old columns
df_activities_exp = df_activities_exp.drop(
    ["Fecha de vencimiento", "Hora de vencimiento"], axis=1
)

### Pivoting

#### Pivoting "df_activities_com"

In [7]:
# defining meetings done and canceled
R1_done = [
    'R1'
]

R2_done = [
    'R2 online', 
    'R2 In', 
    'R2 Out'
]

R1_canceled = [
    'R1 cancelado' 
]

R2_canceled = [
    'R2 in cancelado', 
    'R2 out cancelado',
    'R2 online cancelado'
]

# defining calls effectives and not effectives

effectives_calls = [
    'Llamada de prospección efectiva', 
    'Llamada de seguimiento efectiva', 
]
non_effectives_calls = [
    'Llamada de prospección No efectiva', 
    'Llamada de seguimiento no efectiva',
    'Llamada sin avance'
]

# defining WhatsApp activities
wa_activities = [
    'Seguimiento WSP',
    'Videowsp'
]

# defining total activities in COM
calls = effectives_calls + non_effectives_calls 
meetings_done = R1_done + R2_done
meetings_canceled = R1_canceled + R2_canceled
activities = calls + meetings_done + meetings_canceled + wa_activities


# new count columns 
column_mappings = {
    'Total Actividades': activities,
    'Conteo Llamadas': calls,
    'Llamadas Efectivas': effectives_calls,
    'Llamadas No Efectivas': non_effectives_calls,
    'Contacto WA': wa_activities,
    'Reuniones Hechas': meetings_done,
    'Reuniones Canceladas': meetings_canceled,
    'Conteo R1': R1_done,
    'Conteo R2': R2_done,
    'Conteo R1 Canceladas' : R1_canceled,
    'Conteo R2 Canceladas' : R2_canceled
    }

for column, activity_list in column_mappings.items():
    df_activities_com[column] = df_activities_com['Tipo'].isin(activity_list).astype(int)

# count activities 
count_per_deal = df_activities_com.groupby('Organización').agg(
    Total_Actividades=('Tipo', 'count'),
    Total_Llamadas=('Conteo Llamadas', 'sum'),
    Llamadas_Efectivas=('Llamadas Efectivas', 'sum'),
    Llamadas_No_Efectivas=('Llamadas No Efectivas', 'sum'),
    WA_Seguimiento=('Contacto WA', 'sum'),
    Reuniones_Hechas = ('Reuniones Hechas', 'sum'),
    Reuniones_Canceladas = ('Reuniones Canceladas', 'sum'),
    Reunion1_Hechas = ('Conteo R1', 'sum'),
    Reunion2_Hechas = ('Conteo R2', 'sum'),
    Reunion1_Canceladas = ('Conteo R1 Canceladas', 'sum'),
    Reunion2_Canceladas = ('Conteo R2 Canceladas', 'sum')

).reset_index()

# extracting date for each activity
date_activities = df_activities_com[
    df_activities_com['Tipo'].isin(activities)
].pivot_table(
    index='Organización',
    columns='Tipo',
    values='Hora en que se marcó como completada',
    aggfunc=lambda x: ', '.join(x) # combine all dates between commas
).reset_index()

# rename columns
date_activities.columns.name = None
date_activities.rename(columns=lambda x: f"Fecha {x}" if x != 'Organización' else x, 
                       inplace=True
                       )

# Combinar ambos resultados
df_pivoted_com = pd.merge(
    count_per_deal, 
    date_activities,
    on='Organización',
    how='left'
)


#### Pivoting "df_activities_exp"

In [8]:
# defining activities done and canceled

kickoff_done = [
    'Kick Off'
]

kickoff_canceled = [
    'Kick Off cancelada'
]


training_done = [
    'Capacitación ERP'
]

training_canceled = [
    'Capacitación ERP cancelada'
]

# defining calls effectives and non effectives
effectives_calls = [
    'Llamada de seguimiento efectiva'
]

non_effectives_calls = [
    'Llamada de seguimiento no efectiva'
]

# defining WhatsApp activities
wa_activities = [
    'Seguimiento WSP',
    'Videowsp'
]

# defining total activities in EXP
calls = effectives_calls + non_effectives_calls 
kickoff_done = kickoff_done
kickoff_canceled = kickoff_canceled
trainings_done = training_done
trainings_canceled = training_canceled
activities = calls + kickoff_done + kickoff_canceled + trainings_done + trainings_canceled + wa_activities

# new count columns 
column_mappings = {
    'Total Actividades': activities,
    'Conteo Llamadas': calls,
    'Llamadas Efectivas': effectives_calls,
    'Llamadas No Efectivas': non_effectives_calls,
    'Contacto WA': wa_activities,
    'Conteo Kick Off Hechas' : kickoff_done,
    'Conteo Kick Off Canceladas' : kickoff_canceled,
    'Conteo Capacitaciones Hechas': trainings_done,
    'Conteo Capacitaciones Canceladas': trainings_canceled,
}

# Crear las nuevas columnas usando un bucle
for column, activity_list in column_mappings.items():
    df_activities_exp[column] = df_activities_exp['Tipo'].isin(activity_list).astype(int)


# count activities 
count_per_deal = df_activities_exp.groupby('Organización').agg(
    Total_Actividades=('Tipo', 'count'),
    Total_Llamadas=('Conteo Llamadas', 'sum'),
    Llamadas_Efectivas=('Llamadas Efectivas', 'sum'),
    Llamadas_No_Efectivas=('Llamadas No Efectivas', 'sum'),
    WA_Seguimiento=('Contacto WA', 'sum'),
    Kickoff_Hechas=('Conteo Kick Off Hechas','sum'),
    Kickoff_Canceladas=('Conteo Kick Off Canceladas', 'sum'),
    Capacitaciones_Hechas=('Conteo Capacitaciones Hechas', 'sum'),
    Capacitaciones_Canceladas=('Conteo Capacitaciones Canceladas', 'sum')
).reset_index()

# extracting date for each activity
date_activities = df_activities_exp[
    df_activities_exp['Tipo'].isin(activities)
].pivot_table(
    index='Organización',
    columns='Tipo',
    values='Hora en que se marcó como completada',
    aggfunc=lambda x: ', '.join(x) # combine all dates between commas
).reset_index()

# rename columns
date_activities.columns.name = None
date_activities.rename(columns=lambda x: f"Fecha {x}" if x != 'Organización' else x, 
                       inplace=True
                       )

# Combinar ambos resultados
df_pivoted_exp = pd.merge(
    count_per_deal,
    date_activities, 
    on='Organización', 
    how='left'
)

#### Merged Dataframes

In [9]:
# merge of deals and activities from comercial
df_com_merged = pd.merge(df_pivoted_com, df_deals_com, 
                      left_on='Organización', 
                      right_on='org_id.value', 
                      # how='right'
                      )

In [10]:
# merge of deals and activities from comercial
df_exp_merged = pd.merge(df_pivoted_exp, df_deals_exp, 
                      left_on='Organización', 
                      right_on='org_id.value', 
                      # how='right'
                      )

In [11]:
df_com = df_com_merged.copy()
df_exp = df_exp_merged.copy()

#### Comercial Enriched

##### Split grouped date

In [12]:
dates_to_split = [
    ('Fecha Llamada de prospección efectiva', 'Llamada de prospección efectiva'),
    ('Fecha Llamada de prospección No efectiva', 'Llamada de prospección no efectiva'),
    ('Fecha Llamada de seguimiento efectiva', 'Llamada de seguimiento efectiva'),
    ('Fecha Llamada de seguimiento no efectiva', 'Llamada de seguimiento no efectiva'),
    ('Fecha Llamada sin avance', 'Llamada sin avance'),
    ('Fecha Seguimiento WSP', 'Seguimiento WSP'),
    ('Fecha Videowsp', 'Video WSP'),
    ('Fecha R1', 'R1'), 
    ('Fecha R1 cancelado', 'R1 Cancelado'), 
    ('Fecha R2 In', 'R2 In'), 
    ('Fecha R2 Out','R2 Out'),
    ('Fecha R2 online', 'R2 Online'),
    ('Fecha R2 in cancelado', 'R2 In Cancelado'),
    ('Fecha R2 out cancelado', 'R2 Out Cancelado'),
    ('Fecha R2 online cancelado', 'R2 Online Cancelado')
]

for original_column, new_prefix in dates_to_split:
    df_com = split_column(df_com, original_column, new_prefix)

In [13]:
# convert to datetime
prefix_dates_com = [
    'Negocio creado el', 
    'Fecha de ganado', 
    'Fecha de cierre prevista'
]

prefix_calls_com = [
    'Llamada de prospección efectiva', 
    'Llamada de prospección no efectiva', 
    'Llamada de seguimiento efectiva', 
    'Llamada de seguimiento no efectiva', 
    'Llamada sin avance', 
    'Seguimiento WSP', 
    'Video WSP', 
]
prefix_meetings_com = [
    'R1', 
    'R1 Cancelado',
    'R2'
]

prefix_com = prefix_dates_com + prefix_calls_com + prefix_meetings_com

df_com = process_date_columns(df_com, prefix_com)

##### Calculate New Columns

In [14]:
# finding the kind of the first contact, also the date
prefix_and_cat = {
    'Llamada de prospección efectiva': 'Efectiva',
    'Llamada de prospección no efectiva': 'No Efectiva',
    'Llamada sin avance': 'Sin Avance'
}

columns = []
for prefix in prefix_and_cat.keys():
    columns += [col for col in df_com.columns if col.startswith(prefix)]

columns = list(set(columns))

# Null Values to ''
df_com[columns] = df_com[columns].fillna('')

df_com['Primer Contacto'] = df_com[columns].min(axis=1)

df_com['Tipo Primer Contacto'] = df_com.apply(contact_type, axis=1, col_name='Primer Contacto', prefix_and_cat=prefix_and_cat)



C:\Users\DELL\AppData\Local\Temp\ipykernel_15608\1186315701.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_com['Primer Contacto'] = df_com[columns].min(axis=1)
C:\Users\DELL\AppData\Local\Temp\ipykernel_15608\1186315701.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_com['Tipo Primer Contacto'] = df_com.apply(contact_type, axis=1, col_name='Primer Contacto', prefix_and_cat=prefix_and_cat)


In [15]:
# finding the kind of the first meeting, also the date
prefix_and_cat = {
    'R1': 'Hecha',
    'R1 Cancelado': 'Cancelada'
}

columns = []
for prefix in prefix_and_cat.keys():
    columns += [col for col in df_com.columns if col.startswith(prefix)]

columns = list(set(columns))

# Null Values to ''
df_com[columns] = df_com[columns].fillna('')

df_com['Primera Reunión'] = df_com[columns].min(axis=1)

df_com['Tipo Primera Reunión'] = df_com.apply(contact_type, axis=1, col_name='Primera Reunión', prefix_and_cat=prefix_and_cat)

C:\Users\DELL\AppData\Local\Temp\ipykernel_15608\3669088385.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_com['Primera Reunión'] = df_com[columns].min(axis=1)
C:\Users\DELL\AppData\Local\Temp\ipykernel_15608\3669088385.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_com['Tipo Primera Reunión'] = df_com.apply(contact_type, axis=1, col_name='Primera Reunión', prefix_and_cat=prefix_and_cat)


In [16]:
# finding the kind of the second meeting, also the date
prefix_and_cat = {
    'R2 In': 'Hecha',
    'R2 Out': 'Hecha',
    'R2 Online' : 'Hecha',
    'R2 In Cancelado': 'Cancelada',
    'R2 Out Cancelado': 'Cancelada',
    'R2 Online Cancelado': 'Cancelada'
}

columns = []
for prefix in prefix_and_cat.keys():
    columns += [col for col in df_com.columns if col.startswith(prefix)]

columns = list(set(columns))

# Null Values to ''
df_com[columns] = df_com[columns].fillna('')

df_com['Segunda Reunión'] = df_com[columns].min(axis=1)

df_com['Tipo Segunda Reunión'] = df_com.apply(contact_type, axis=1, col_name='Segunda Reunión', prefix_and_cat=prefix_and_cat)

C:\Users\DELL\AppData\Local\Temp\ipykernel_15608\2520629443.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_com['Segunda Reunión'] = df_com[columns].min(axis=1)
C:\Users\DELL\AppData\Local\Temp\ipykernel_15608\2520629443.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_com['Tipo Segunda Reunión'] = df_com.apply(contact_type, axis=1, col_name='Segunda Reunión', prefix_and_cat=prefix_and_cat)


In [17]:
# 'Tiempo DealCreado - Primer Contacto'
df_com = time_calculator_min(df_com, 'Negocio creado el', 'Primer Contacto')


c:\UMSS 2024\experiencia\notebooks\../scripts\silver_functions.py:104: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[end_date]-df[start_date]


In [18]:
# 'Tiempo Negocio creado el-Primer Contacto'
df_com['Rango de Contacto'] = np.select(
    [
        df_com['Tiempo Negocio creado el-Primer Contacto (min)'].eq(''),  
        df_com['Tiempo Negocio creado el-Primer Contacto (min)'].isna(),  
        df_com['Tiempo Negocio creado el-Primer Contacto (min)'] < 6  
    ],
    [
        'Sin Llamada de primer contacto', 
        'Sin Llamada de primer contacto', 
        'Dentro del rango' 
    ],
    default='Fuera del rango'  
)

# We take into account the 6 minutes as a limit, resulting from the previously performed contact analysis

C:\Users\DELL\AppData\Local\Temp\ipykernel_15608\866244994.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_com['Rango de Contacto'] = np.select(


##### Category Structural Break Calculation

In [19]:
# New Categories

date = pd.Timestamp('2024-04-01')

df_com['New Categories'] = np.where(
    df_com['Negocio creado el'] > date,  
    1,  
    0 
    )

C:\Users\DELL\AppData\Local\Temp\ipykernel_15608\3052524186.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_com['New Categories'] = np.where(


Crossing Data

In [20]:
df_com['R1'] = np.where(
    df_com['Tipo Primera Reunión'] == 'Hecha',
    1,  
    0   
)

C:\Users\DELL\AppData\Local\Temp\ipykernel_15608\4004547658.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_com['R1'] = np.where(


In [21]:
df_com['R2'] = np.where(
    df_com['Tipo Segunda Reunión'] == 'Hecha',
    1,  
    0   
)

C:\Users\DELL\AppData\Local\Temp\ipykernel_15608\1420537603.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_com['R2'] = np.where(


In [22]:
df_com['R1yR2'] = np.where((df_com['R1'] == 1) & (df_com['R2'] == 1), 1, 0)

C:\Users\DELL\AppData\Local\Temp\ipykernel_15608\4107020047.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_com['R1yR2'] = np.where((df_com['R1'] == 1) & (df_com['R2'] == 1), 1, 0)


In [23]:
df_subset = df_com[['R1', 'R2', 'R1yR2']]

In [24]:
df_subset.value_counts()

R1  R2  R1yR2
0   1   0        498
    0   0         98
1   1   1         70
Name: count, dtype: int64

##### Selecting Columns

In [25]:
# creating a copy from df_com
df_com_metrics = df_com.copy()

In [26]:
columns = [
    'org_id.value',
    'Tipo de cliente',  
    'Total_Actividades',  
    'Total_Llamadas', 
    'Llamadas_Efectivas', 
    'Llamadas_No_Efectivas', 
    'WA_Seguimiento',
    'Reuniones_Hechas', 
    'Reuniones_Canceladas',     
    'Tipo Primer Contacto',  
    'Rango de Contacto', 
    'New Categories',
    'R1yR2'
    ]

df_com_metrics = df_com_metrics[columns]


#### Experience Enriched

##### Split Grouped date

In [27]:
dates_to_split = [
    ('Fecha Llamada de seguimiento efectiva', 'Llamada de seguimiento efectiva'),
    ('Fecha Llamada de seguimiento no efectiva', 'Llamada de seguimiento no efectiva'),
    ('Fecha Seguimiento WSP', 'Seguimiento WSP'),
    ('Fecha Kick Off', 'Kick Off'), 
    ('Fecha Kick Off cancelada', 'Kick Off cancelada'), 
    ('Fecha Capacitación ERP', 'Capacitación ERP'),
    ('Fecha Capacitación ERP cancelada', 'Capacitación ERP cancelada')
]

for original_column, new_prefix in dates_to_split:
    df_exp = split_column(df_exp, original_column, new_prefix)

In [28]:
# convert to datetime
prefix_dates_exp = [
    'Negocio creado el', 
    'Fecha de ganado', 
    'Fecha de perdido',
    'Fecha de cierre prevista',
    '(EXP) Fecha Kickoff', 
    '(EXP) Fecha de finalización de onboarding',
    '(C)(EXP) Fecha Suscripción Inicio'
]

prefix_calls_exp = [
    'Llamada de seguimiento efectiva', 
    'Llamada de seguimiento no efectiva', 
    'Seguimiento WSP', 
    'Video WSP',     
]

prefix_meetings_exp = [
    'Kick Off',
    'Kick Off cancelada',
    'Capacitación ERP',
    'Capacitación ERP cancelada'
]

prefix_exp = prefix_dates_exp + prefix_calls_exp + prefix_meetings_exp

df_exp = process_date_columns(df_exp, prefix_exp)

c:\UMSS 2024\experiencia\notebooks\../scripts\silver_functions.py:78: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')


##### Calculate New Columns

In [29]:
# finding the kind of the first contact, also the date
prefix_and_cat = {
    'Llamada de seguimiento efectiva': 'Efectiva',
    'Llamada de seguimiento no efectiva': 'No Efectiva',
}

columns = []
for prefix in prefix_and_cat.keys():
    columns += [col for col in df_exp.columns if col.startswith(prefix)]

columns = list(set(columns))

# Null Values to ''
df_exp[columns] = df_exp[columns].fillna('')

df_exp['Primer Seguimiento'] = df_exp[columns].min(axis=1)

df_exp['Tipo Primer Seguimiento'] = df_exp.apply(contact_type, axis=1, col_name='Primer Seguimiento', prefix_and_cat=prefix_and_cat)


In [30]:
# subsetting a data frame
df_com_subset = df_com[['org_id.value', 'Fecha de ganado']]

# Realizar el merge
df_exp = df_exp.merge(df_com_subset, on='org_id.value', how='left')

In [31]:
# subsetting a data frame
df_exp_subset = df_exp[['org_id.value', '(EXP) Fecha Kickoff', '(EXP) Fecha de finalización de onboarding']]

# Realizar el merge
df_exp = df_exp.merge(df_exp_subset, on='org_id.value', how='left')

In [32]:
df_exp['(EXP) Fecha Kickoff_y'] = pd.to_datetime(df_exp['(EXP) Fecha Kickoff_y'])

In [33]:
# finding the kind of 
df_exp['Onboarding'] = np.where(
    df_exp['(EXP) Fecha de finalización de onboarding_y'].isna(), 
    'No Finalizado',
    'Finalizado'
)

In [34]:
df_exp['(EXP) Fecha Kickoff_y'] = df_exp['(EXP) Fecha Kickoff_y'] + pd.Timedelta(hours=23, minutes=59, seconds=59)
# aumentamos la ultima hora del dia para que al comparar no se confunda como el primero en ocurrir

# finding the kind of the first training, also the date
prefix_and_cat = {
    '(EXP) Fecha Kickoff': 'Efectiva',
    'Capacitación ERP': 'Cancelada',
}

columns = []
for prefix in prefix_and_cat.keys():
    columns += [col for col in df_exp.columns if col.startswith(prefix)]

columns = list(set(columns))


# Null Values to ''
df_exp[columns] = df_exp[columns].fillna('')

df_exp['Kick Off'] = df_exp[columns].min(axis=1)

df_exp['Tipo Kick Off'] = df_exp.apply(contact_type, axis=1, col_name='Kick Off', prefix_and_cat=prefix_and_cat)

In [35]:
# finding the kind of the first training, also the date
prefix_and_cat = {
    'Capacitación ERP': 'Hecha',
    'Capacitación ERP cancelada': 'Cancelada',
}

columns = []
for prefix in prefix_and_cat.keys():
    columns += [col for col in df_exp.columns if col.startswith(prefix)]

columns = list(set(columns))

# Null Values to ''
df_exp[columns] = df_exp[columns].fillna('')

df_exp['Primera Capacitación'] = df_exp[columns].min(axis=1)

df_exp['Tipo Primera Capacitación'] = df_exp.apply(contact_type, axis=1, col_name='Primera Capacitación', prefix_and_cat=prefix_and_cat)

#### Label Calculation

In [36]:
# 'Tiempo Negocio creado el - Fecha de perdido'
df_exp = time_calculator_days(df_exp, 'Fecha de ganado_y', 'Fecha de perdido') 

In [37]:
df_exp['Fecha de perdido']. info()

<class 'pandas.core.series.Series'>
RangeIndex: 539 entries, 0 to 538
Series name: Fecha de perdido
Non-Null Count  Dtype         
--------------  -----         
169 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 4.3 KB


In [38]:
# 'Tiempo Negocio creado el-Fecha de perdido (días)'
df_exp['Churn Comercial'] = df_exp['Tiempo Fecha de ganado_y-Fecha de perdido (días)'].apply(
    lambda x: 1 if x < 90 and pd.notna(x) else 0
)

# We take into account a limit of 90 days for it to be considered 'Commercial Churn' 
# according to parameters previously defined in tuGerente.com

##### Selecting Columns

In [39]:
# creating a copy from df_exp
df_exp_metrics = df_exp.copy()

In [40]:
columns = [
    'org_id.value',
    'Total_Actividades',
    'Total_Llamadas',
    'Llamadas_Efectivas',
    'Llamadas_No_Efectivas',
    'WA_Seguimiento',
    'Kickoff_Hechas',
    'Kickoff_Canceladas',
    'Capacitaciones_Hechas',
    'Capacitaciones_Canceladas',
    'Tipo Primera Capacitación',
    'Onboarding',
    '(C) (EXP) Plazo y Pago',
    'Churn Comercial'
]

df_exp_metrics = df_exp_metrics[columns]

### Export Enriched Data

In [41]:
# final merged
df_contact_metrics = pd.merge(df_com_metrics, df_exp_metrics, 
                      on='org_id.value', 
                      suffixes=('_com', '_exp')
                      )

In [42]:
# Export DataFrame to CSV - CONTACT 
df_contact_metrics.to_csv(r'..\data\output_silver\03_silver_enriched\contact_metrics.csv', index=False)
df_com_metrics.to_csv(r'..\data\output_silver\03_silver_enriched\contact_metrics_com.csv', index=False)
df_exp_metrics.to_csv(r'..\data\output_silver\03_silver_enriched\contact_metrics_exp.csv', index=False)